## Day 17 - Heat loss on a locomotive

**Part 1: Find a path that minimises heat loss from top left corner of grid to bottom right corner, can never go in a straight line for more than 3 blocks! - notably this doesn't include if you're e.g. turning 'up' and then going three to the right, the up didn't count as going to the right**

Have never done path finding before... not looking forward to this!

In [71]:
with open("./example.txt") as f:
    example_lines = f.read().splitlines()

with open("./input.txt") as f:
    input_lines = f.read().splitlines()

example_lines

['2413432311323',
 '3215453535623',
 '3255245654254',
 '3446585845452',
 '4546657867536',
 '1438598798454',
 '4457876987766',
 '3637877979653',
 '4654967986887',
 '4564679986453',
 '1224686865563',
 '2546548887735',
 '4322674655533']

In [72]:
from typing import Tuple, Iterator

GridLocation = Tuple[int, int]

class SquareGrid:
    def __init__(self, width: int, height: int):
        self.width = width
        self.height = height
    
    def in_bounds(self, id: GridLocation) -> bool:
        (x, y) = id
        return 0 <= x < self.width and 0 <= y < self.height
    
    def neighbors(self, id: GridLocation) -> Iterator[GridLocation]:
        (x, y) = id
        neighbors = [(x+1, y), (x-1, y), (x, y-1), (x, y+1)] # E W N S
        # see "Ugly paths" section for an explanation:
        if (x + y) % 2 == 0: neighbors.reverse() # S N W E
        results = filter(self.in_bounds, neighbors)
        return results

In [73]:
class GridWithWeights(SquareGrid):
    def __init__(self, width: int, height: int, weights: list[list[int]]):
        super().__init__(width, height)
        self.weights: dict[GridLocation, int] = {
            (c, r): val
            for c, col in enumerate(weights)
            for r, val in enumerate(col) 
        }
    
    def cost(self, to_node: GridLocation, amount_travelled_in_direction: int = 0) -> float:
        return self.weights.get(to_node, 1) + 0 if amount_travelled_in_direction < 3 else 1e6

In [74]:
from queue import PriorityQueue

In [75]:
from typing import Optional

def dijkstra_search(grid: GridWithWeights, start: GridLocation, goal: GridLocation):

    manual_track_last_four = []

    frontier = PriorityQueue()
    frontier.put(start, 0)
    came_from: dict[GridLocation, Optional[GridLocation]] = {}
    cost_so_far: dict[GridLocation, int] = {}
    came_from[start] = None
    cost_so_far[start] = 0
    
    while not frontier.empty():
        current: GridLocation = frontier.get()
        
        if current == goal:
            break
        
        for next in grid.neighbors(current):

            came_from_previous = [val for val in came_from.values()]
            came_from_previous.reverse()
            came_from_previous = came_from_previous[:4]
            if len(came_from_previous) < 4 or None in came_from_previous:
                can_continue_forward = True
            else:
                loc0 = came_from_previous[0]
                y0, x0 = loc0
                loc1 = came_from_previous[1]
                y1, x1 = loc1
                loc2 = came_from_previous[2]
                y2, x2 = loc2
                loc3 = came_from_previous[3]
                y3, x3 = loc3
                can_continue_forward = not (
                    (y1-y0 == y2-y1 == y3-y2) or (x1-x0 == x2-x1 == x3-x2)
                )
            
            if can_continue_forward:
                travelled_ = 1
                # not_allowed_y, not_allowed_x = 9999, 9999
            else:
                travelled_ = 5
                # if ( (diff:= y1-y0) == y2-y1 == y3-y2):
                #     not_allowed_y = y0 + diff
                #     not_allowed_x = 99999
                # elif ( (diff:= x1-x0) == x2-x1 == x3-x2):
                #     not_allowed_y = 99999
                #     not_allowed_x = x0 + diff
                
            
            # next_y, next_x = next
            # print(next_y, next_x, "|", not_allowed_y, not_allowed_x)
            # print(travelled_)
            new_cost = cost_so_far[current] + grid.cost(next, travelled_)
            

            if (next not in cost_so_far or new_cost < cost_so_far[next]):
                cost_so_far[next] = new_cost
                priority = new_cost
                frontier.put(next, priority)
                came_from[next] = current
    
    return came_from, cost_so_far

In [76]:
def reconstruct_path(came_from: dict[GridLocation, GridLocation],
                     start: GridLocation, goal: GridLocation) -> list[GridLocation]:

    current: GridLocation = goal
    path: list[GridLocation] = []
    if goal not in came_from: # no path was found
        return []
    while current != start:
        path.append(current)
        current = came_from[current]
    path.append(start) # optional
    path.reverse() # optional
    return path

In [78]:
# def part1(lines: list[str]) -> int:
    # grid = [list(map(int, [*line])) for line in lines]
    
#     # such that grid[y][x] takes you to (y,x) where (0,0) is top left
#     goal = (len(grid) - 1, len(grid[0]) - 1)

#     square_grid = GridWithWeights(width=len(grid[0]), height=len(grid), weights=grid)
#     came_from, cost_so_far = dijkstra_search(square_grid, (0,0), goal)

#     print(came_from)

#     return reconstruct_path(came_from, (0,0), goal)
    
    
    

# part1(example_lines)

idk that was a lot of code from some blog and idk if it's adaptable for this

lets try ourselves a less sophisticated approach

In [115]:

import collections
def find_paths(grid, start_coord):
    paths       = set()  # paths will be added here
    state_queue = collections.deque([]) # Pending states which have not been explored yet
    # State is a tuple consists of:
    # 1. current coordinate
    # 2. crack code path
    # 3. set of visited coordinates in that path
    state       = [start_coord, [], {start_coord}]  # Starting state
    while True:
        # Getting all possible neighboring states
        # Crack code (right=0, down=1, left=2, up=3)

        state_right = [(state[0][0],state[0][1]+1), state[1] + [0], state[2].copy()] if (state[0][1]+1 < len(grid[state[0][0]]) and (len(state[1]) == 0 or not all(s==0 if len(state[1]) >= 3 else False for s in state[1][-3:]))) else None
        state_down  = [(state[0][0]+1,state[0][1]), state[1] + [1], state[2].copy()] if (state[0][0]+1 < len(grid) and (len(state[1]) == 0 or not all(s==1 if len(state[1]) >= 3 else False for s in state[1][-3:]))) else None
        state_left  = [(state[0][0],state[0][1]-1), state[1] + [2], state[2].copy()] if (state[0][1]-1 >= 0 and (len(state[1]) == 0 or not all(s==2 if len(state[1]) >= 3 else False for s in state[1][-3:]))) else None
        state_up    = [(state[0][0]-1,state[0][1]), state[1] + [3], state[2].copy()] if (state[0][0]-1 >= 0 and (len(state[1]) == 0 or not all(s==3 if len(state[1]) >= 3 else False for s in state[1][-3:]))) else None
        # Adding to the queue all the unvisited states, as well as to the visited to avoid returning to states

        blocked_counter = 0
        for next_state in [state_right, state_down, state_left, state_up]:
            if next_state is None:
                blocked_counter += 1
            elif next_state[0] in state[2] or grid[next_state[0][0]][next_state[0][1]] == 0:
                blocked_counter += 1
            else:
                next_state[2].add(next_state[0])
                state_queue.append(next_state)
                
        # After checking all directions' if reached a 'dead end', adding this path to the path set
        if blocked_counter == 4:
            paths.add(tuple(state[1]))
        # Popping next state from the queue and updating the path if needed
        try:
            state = state_queue.pop()
        except IndexError:
            break
    return paths

In [117]:
# def part1(lines: list[str]) -> int:
#     grid = [list(map(int, [*line])) for line in lines]

#     paths = find_paths(grid, (0,0))
#     return paths

# part1(example_lines)


# ugh that's just too inefficient isnt it obviously

In [129]:
# lets follow along Neutrino but try and learn since this is completely foreign lol. As long as I understand everything I think is ok.
from heapq import heappush, heappop 
def part1(lines: list[str]) -> int:
    grid = [list(map(int, [*line])) for line in lines]
    
    # dijkstra's algorithm

    seen = set()

    # EACH TUPLE IS OF THE FORM
    # (HEATLOSS, ROW, COL, DELTA_R, DELTA_C, NUM_STEPS_IN_DIRECTION)
    priority_queue = [(0, 0, 0, 0, 0, 0)]

    while priority_queue:
        heat_loss, r, c, dr, dc, dir_steps = heappop(priority_queue)

        # if at destination we want to break!
        if r == len(grid) - 1 and c == len(grid[0]) - 1:
            return heat_loss
            # Since we're using a priority queue here, we always get minimal values of heat_loss out the way first, so we cannot get here without it being the optimal value!

        if (r, c, dr, dc, dir_steps) in seen:
            # done it before
            continue
            
        seen.add((r, c, dr, dc, dir_steps))
        
        if dir_steps < 3 and (dr, dc) != (0, 0):
            # if we haven't gone in same direction for 3 and aren't standing still

            # nr = next_r
            nr = r + dr
            nc = c + dc
            if 0 < nr < len(grid) and 0 < nc < len(grid[0]):
                heappush(priority_queue, (heat_loss + grid[nr][nc], nr, nc, dr, dc, dir_steps + 1))
        
        # can also TURN!
        for ndr, ndc in [(0,1), (1,0), (-1, 0), (0, -1)]:
            if (ndr, ndc) != (dr, dc) and (ndr, ndc) != (-dr, -dc):
                # can't be current direction or backwards!
                nr = r + ndr
                nc = c + ndc
                if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
                    heappush(priority_queue, (heat_loss + grid[nr][nc], nr, nc, ndr, ndc, 1))

assert part1(example_lines) == 108
part1(input_lines)

886

**Part 2: Now using big train that go fast but not for long, i.e. have to go 4 continous steps in the same direction, but no more than 10!**

obvs doing this bit alone no follow along now that I understand the algorithm, kinda brilliant tbf, Dijkstra was really onto something with this one

In [136]:
with open("./example2.txt") as f:
    example2_lines = f.read().splitlines()

In [143]:
def part2(lines: list[str]) -> int:
    grid = [list(map(int, [*line])) for line in lines]
    
    # dijkstra's algorithm

    seen = set()

    # EACH TUPLE IS OF THE FORM
    # (HEATLOSS, ROW, COL, DELTA_R, DELTA_C, NUM_STEPS_IN_DIRECTION)

    # gonna add all the starting directions just to keep it simple
    priority_queue = [
        (0, 0, 0, 1, 0, 1),
        (0, 0, 0, 0, 1, 1),
    ]

    while priority_queue:
        heat_loss, r, c, dr, dc, dir_steps = heappop(priority_queue)

        # if at destination we want to break!
        # missed the condition that has to have been travelling for 4 at the end to stop also!
        if r == len(grid) - 1 and c == len(grid[0]) - 1 and dir_steps >= 4:
            return heat_loss
            # Since we're using a priority queue here, we always get minimal values of heat_loss out the way first, so we cannot get here without it being the optimal value!

        if (r, c, dr, dc, dir_steps) in seen:
            # done it before
            continue
            
        seen.add((r, c, dr, dc, dir_steps))
        
        if dir_steps < 10:
            # if we haven't gone in same direction for 10 yet and aren't standing still

            # nr = next_r
            nr = r + dr
            nc = c + dc
            if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
                heappush(priority_queue, (heat_loss + grid[nr][nc], nr, nc, dr, dc, dir_steps + 1))
        
        # can also TURN!
        for ndr, ndc in [(0,1), (1,0), (-1, 0), (0, -1)]:
            if (ndr, ndc) != (dr, dc) and (ndr, ndc) != (-dr, -dc) and (dir_steps >= 4):
                # can't be current direction or backwards!
                nr = r + ndr
                nc = c + ndc
                if 0 <= nr < len(grid) and 0 <= nc < len(grid[0]):
                    heappush(priority_queue, (heat_loss + grid[nr][nc], nr, nc, ndr, ndc, 1))

assert part2(example_lines) == 94
assert part2(example2_lines) == 71
part2(input_lines)

1055

Very educational day !